In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re

#for the cartesian product
import itertools
from itertools import product

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage
from scipy.ndimage.filters import gaussian_filter

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

##550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov_test.root");en="KE=11.5e-550eV";CH="CH11";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov_test.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov_test.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov_test.root")

# #546eV - 8.2eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_10800-2350ns_multiCH11.root");en="KE=8.2-546eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CL_10800-2350ns_multiCH11.root")

# 544eV - 6.1eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH11.root");en="KE=6.1-544eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH11.root")

# #542eV - 4eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_10800-2350ns_COMBI_multiCH11.root");en="KE=4-542eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CL_10800-2350ns_COMBI_multiCH11.root")

# #541eV 3eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_10800-2350ns_multiCH11.root");en="KE=3-541eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CL_10800-2350ns_multiCH11.root")

#uncomment this for the special case!
# CH="CH12"
path = "angular_distr_el/"+CH+"/"

enant="R"
n2=False
gaussian=0.6

#move first the ID you need
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/", "ID4_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

loc=path+IDs[0]+dirs[0]

#to alow the helicity comparison without the S enantiomer
if en != "550eV":
    fileSCR = None
    fileSCL = None

#new uproot4
# https://uproot.readthedocs.io/en/latest/basic.html
d=dict(fileRCR[loc].classnames()) # shows the type of each item
# k=fileRCR[loc].keys() #same as .classnames()
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
d1=type(check)
check.all_members; #all members of the single histogram

#set the gaussian for the smoothing before PECD and/or contrast maps

In [ ]:
#Loading of the MFPADS and cos len = 27
MFPAD_RCR,ctheta_RCR,ctheta_RCR_err,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_import_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL,ctheta_RCL_err=fugi.import_MFPAD(fileRCL, loc)
#projection of cos on x axis: len = 18
cosx_RCR=fugi.projection(MFPAD_RCR,alongaxis=0)
cosx_RCL=fugi.projection(MFPAD_RCL,alongaxis=0)

if fileSCL and fileSCR is not None:    
    MFPAD_SCR,ctheta_SCR,ctheta_SCR_err=fugi.import_MFPAD(fileSCR, loc)
    MFPAD_SCL,ctheta_SCL,ctheta_SCL_err=fugi.import_MFPAD(fileSCL, loc)
    #projection of cos on x axis
    cosx_SCR=fugi.projection(MFPAD_SCR,alongaxis=0)
    cosx_SCL=fugi.projection(MFPAD_SCL,alongaxis=0)

In [ ]:
##########################b1 map x,y coordiantes###############################
"""
PHI - CTHETA photon with respect to the LF (in sperical coordinates)
NOTE the go coordiantes are organised accordinf to the SORTED MFPAD, therefore cos(theta)=[-1,1]
REDUCED=TRUE => ADJUSTED TO EXPERIMENT: -0.835,0.835,-165,165
REDUCED=FALSE => ADJUSTED TO THEORY: -1,1,-180,180
"""
# fugi = reload(fugi)

xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, cos_lin=True, source=True)
xgo_cos,ygo_cos,cosphi_adj_cos = fugi.create_gocoords(a=1, cos_lin=True, source=True)

# list(zip(xgo_phi,ygo_phi))
print("(xgo_cos (phi), ygo_cos (ctheta)) \n", list(zip(xgo_cos,ygo_cos))[:6])
print("\n cosphi_adj_cos  \n", cosphi_adj_cos[:6])
print("\n cosphi_adj_phi \n", cosphi_adj_phi[:6])

In [ ]:
#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_import_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #cos = (18,), MFPAD_axis = (19,)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape, ", cosM[0] = ",cosM[0])

#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #COS = (18,), MFPAD_axis = (19,)
phiMM, cosMM = np.meshgrid(phiM, cosM)

phicos_phi=list(product(phiM, cosM))
phi_full = np.array([el[0] for el in phicos_phi])
cos_full = np.array([el[1] for el in phicos_phi])

phi_rad= phi_full*np.pi/180.
theta_rad= np.arccos(cos_full)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("len phi_rad = ", phi_rad.shape)
print("len theta_rad = ", theta_rad.shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

dfind=pd.DataFrame(cosphi_adj_phi, columns=["ctheta","phi"])

In [ ]:
#flipping; axis=1 flips along cos(theta), axis=0 along phi
MFPAD_RCL_flip=[]
MFPAD_RCR_flip=[]

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_RCL:
    MFPAD_RCL_flip.append(np.flip(el, axis=1))
MFPAD_RCL_flip=np.array(MFPAD_RCL_flip)

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_RCR:
    MFPAD_RCR_flip.append(np.flip(el, axis=1))
MFPAD_RCR_flip=np.array(MFPAD_RCR_flip)

if fileSCL and fileSCR is not None:
    MFPAD_SCL_flip=[]
    MFPAD_SCR_flip=[]

    for el in MFPAD_SCL:
        MFPAD_SCL_flip.append(np.flip(el, axis=1))
    MFPAD_SCL_flip=np.array(MFPAD_SCL_flip)

    for el in MFPAD_SCR:
        MFPAD_SCR_flip.append(np.flip(el, axis=1))
    MFPAD_SCR_flip=np.array(MFPAD_SCR_flip)    

In [ ]:
#sum within an experimemt axis=0 is among the 72 elements
#single enantiomer and helicity
#sum all MFPAD: NO NORMALIZATION
MFPAD_RCR_sum=np.sum(MFPAD_RCR, axis=0)
MFPAD_RCL_sum=np.sum(MFPAD_RCL, axis=0)
MFPAD_RCR_flip_sum=np.sum(MFPAD_RCR_flip, axis=0)
MFPAD_RCL_flip_sum=np.sum(MFPAD_RCL_flip, axis=0)
MFPAD_R_sum=np.add(MFPAD_RCR_sum,MFPAD_RCL_sum)
MFPAD_R_sum_flip=np.add(MFPAD_RCR_flip_sum,MFPAD_RCL_flip_sum)

if fileSCL and fileSCR is not None:
    #single enantiomer and helicity
    MFPAD_SCR_sum=np.sum(MFPAD_SCR, axis=0)
    MFPAD_SCL_sum=np.sum(MFPAD_SCL, axis=0)
    MFPAD_SCR_flip_sum=np.sum(MFPAD_SCR_flip, axis=0)
    MFPAD_SCL_flip_sum=np.sum(MFPAD_SCL_flip, axis=0)
    MFPAD_S_sum=np.add(MFPAD_SCR_sum,MFPAD_SCL_sum)
    MFPAD_S_sum_flip=np.add(MFPAD_SCR_flip_sum,MFPAD_SCL_flip_sum)

    if enant == "R":
        #R equivalente MFPAD as sum of two enantiomers: there is no influence of the ellicity
        MFPAD_tot_sum=np.add(MFPAD_R_sum,MFPAD_S_sum_flip) #NOTE S is flipped
        MFPAD_tot_sum_norm=fugi.normalise_with_err(MFPAD_tot_sum,normtype=3)
    else:
        #S equivalente MFPAD as sum of two enantiomers: there is no influence of the ellicity
        MFPAD_tot_sum=np.add(MFPAD_S_sum,MFPAD_R_sum_flip) #NOTE S is flipped
        MFPAD_tot_sum_norm=fugi.normalise_with_err(MFPAD_tot_sum,normtype=3)

else:
    MFPAD_tot_sum=MFPAD_R_sum
    MFPAD_tot_sum_norm=fugi.normalise_with_err(MFPAD_tot_sum,normtype=3)

In [ ]:
#sorting
#let´s work just with NOT NORMALISED becuase of MFPAD
MFPAD_RCR_cos=fugi.sorting_array(MFPAD_RCR, theory=False, a=1)
MFPAD_RCL_cos=fugi.sorting_array(MFPAD_RCL, theory=False, a=1)
MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, theory=False, a=2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, theory=False, a=2)

MFPAD_RCL_flip_cos=fugi.sorting_array(MFPAD_RCL_flip, theory=False, a=1)
MFPAD_RCL_flip_phi=fugi.sorting_array(MFPAD_RCL_flip, theory=False, a=2)
MFPAD_RCR_flip_cos=fugi.sorting_array(MFPAD_RCR_flip, theory=False, a=1)
MFPAD_RCR_flip_phi=fugi.sorting_array(MFPAD_RCR_flip, theory=False, a=2)


if fileSCL and fileSCR is not None:
    MFPAD_SCR_cos=fugi.sorting_array(MFPAD_SCR, theory=False, a=1)
    MFPAD_SCL_cos=fugi.sorting_array(MFPAD_SCL, theory=False, a=1)
    MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, theory=False, a=2)
    MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, theory=False, a=2)

    MFPAD_SCL_flip_cos=fugi.sorting_array(MFPAD_SCL_flip, theory=False, a=1)
    MFPAD_SCL_flip_phi=fugi.sorting_array(MFPAD_SCL_flip, theory=False, a=2)
    MFPAD_SCR_flip_cos=fugi.sorting_array(MFPAD_SCR_flip, theory=False, a=1)
    MFPAD_SCR_flip_phi=fugi.sorting_array(MFPAD_SCR_flip, theory=False, a=2)

In [ ]:
#smart flipping for the single 72 MFPAD image. Has to be in phi because of the structure of reshape, do the cos just as a check
temp_RCLCR=np.empty([12,6,648])
nphi=MFPAD_RCL.shape[1];ncos=MFPAD_RCL.shape[2]

#sum helicities: -cos(theta), phi+180 in photon coordiantes
for j in range(12):
    for i in range(6):
        if j>5:
            temp_RCLCR[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[j-6,5-i])
        else:
            temp_RCLCR[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[6+j,5-i])

MFPAD_RCLCR=np.array(temp_RCLCR).reshape(72,nphi,ncos)
MFPAD_RCLCR_sum=np.sum(MFPAD_RCLCR, axis=0)

#sorting
MFPAD_RCLCR_cos=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=1)
MFPAD_RCLCR_phi=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=2) #equal to the original!
MFPAD_RCLCR_phi_norm=fugi.normalise_with_err(MFPAD_RCLCR_phi)

temp1=[]
for el in temp_RCLCR.reshape(72,nphi,ncos):
    temp1.append(np.flip(el,axis=1))
temp_RCLCR_flip=np.array(temp1).reshape(12,6,-1)

MFPAD_RCLCR_flip=np.array(temp_RCLCR_flip).reshape(72,nphi,ncos)

if fileSCL and fileSCR is not None:
    temp_SCLCR=np.empty([12,6,648])

    #sum helicities: -cos(theta), phi+180 in photon coordiantes
    for j in range(12):
        for i in range(6):
            if j>5:
                temp_SCLCR[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[j-6,5-i])
            else:
                temp_SCLCR[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[6+j,5-i])
    
    MFPAD_SCLCR=np.array(temp_SCLCR).reshape(72,nphi,ncos)
    MFPAD_SCLCR_sum=np.sum(MFPAD_SCLCR, axis=0)

    #sorting
    MFPAD_SCLCR_cos=fugi.sorting_array(MFPAD_SCLCR, theory=False, a=1)
    MFPAD_SCLCR_phi=fugi.sorting_array(MFPAD_SCLCR, theory=False, a=2) #equal to the original!
    MFPAD_SCLCR_phi_norm=fugi.normalise_with_err(MFPAD_SCLCR_phi,normtype=2)

    #sum enantiomer: -cos(theta) fliped axis=1 cos!, -cos(theta) photon coordinates. it shouldn´t be normalized!
    temp_totordered_std=np.empty([12,6,648])
    temp_totordered=np.empty([12,6,648])
    temp_totordered_flip=np.empty([12,6,648]) #to compare to MOx

    for j in range(12):
        for i in range(6):
            temp_totordered_std[j,i]=np.add(temp_SCLCR[j,i],gaussian_filter(temp_RCLCR_flip, sigma=0.3)[j,5-i]) #the correct version to Kilian
            temp_totordered[j,i]=np.add(temp_SCLCR[j,i],gaussian_filter(temp_RCLCR, sigma=0.3)[j,i]) #better for 72 MFPADs and SR PECD? yes
            temp_totordered_flip[j,i]=np.add(gaussian_filter(temp_RCLCR, sigma=0.3)[j,i],np.flip(temp_SCLCR[j,5-i].reshape(nphi,ncos),axis=0).reshape(-1)) #works, but R is noisy

    MFPAD_totordered=np.array(temp_totordered).reshape(72,nphi,ncos)
    MFPAD_totordered_flip=np.array(temp_totordered_flip).reshape(72,nphi,ncos)
    
    #sorting
    MFPAD_totordered_cos=fugi.sorting_array(MFPAD_totordered, theory=False, a=1)
    MFPAD_totordered_phi=fugi.sorting_array(MFPAD_totordered, theory=False, a=2) #equal to the original!
    MFPAD_totordered_phi_norm=fugi.normalise_with_err(MFPAD_totordered_phi,normtype=3)

    MFPAD_RCLCR_sum_norm=fugi.normalise_with_err(MFPAD_RCLCR_sum,normtype=2)
    MFPAD_SCLCR_sum_norm=fugi.normalise_with_err(MFPAD_SCLCR_sum,normtype=2)

    #the cos of the single MFPAD has to be reversed: JUST ALONG COS(theta) -> axis=1
    MFPAD_tot_sum_flip=np.flip(MFPAD_tot_sum,axis=1)
    MFPAD_tot_sum_flip_norm=fugi.normalise_with_err(MFPAD_tot_sum_flip,normtype=2)
    # has to be normalized to avoid the contribution of unballanced counts

else:
    MFPAD_totordered=MFPAD_RCLCR    
    MFPAD_totordered_cos=MFPAD_RCLCR_cos
    MFPAD_totordered_phi=MFPAD_RCLCR_phi

    MFPAD_totordered_flip=np.array(temp1).reshape(72,nphi,ncos)

    #PECD check it is equal to contrast; has to be normalized to avoid the contribution of unballanced counts
    MFPAD_totordered_sum_norm=fugi.normalise_with_err(np.sum(MFPAD_totordered,axis=0),normtype=2)
    MFPAD_totordered_sum_flip_norm=fugi.normalise_with_err(np.sum(MFPAD_totordered_flip,axis=0),normtype=2)

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    ax.text(0,0,str(i))

In [ ]:
dfind["phi"].plot()

In [ ]:
#NOTE change the value manuallyin mgrid.
#theta_rad is lienar, not the cos. This is made for the point cloud rendering, but bisplev requires a linear input from mgrid!
nbin_density=90

cos_new, phi_new = np.mgrid[cos_full.min():cos_full.max():90j, phi_full.min():phi_full.max():90j]

phicos_PHOTON = list(itertools.product(np.linspace(phi_full.min(),phi_full.max(),nbin_density),np.cos(np.linspace(np.arccos(cos_full.min()),np.arccos(cos_full.max()),nbin_density))))

cos_newM=np.array([el[1] for el in phicos_PHOTON])
phi_newM=np.array([el[0] for el in phicos_PHOTON])

theta_rad_new = np.arccos(cos_newM)
phi_rad_new = phi_newM/180.*np.pi

##### full #####
'''theta_newfull, phi_newfull = np.mgrid[-np.pi-0.02:0.-0.02:90j, -179.9:179.9:90j]
cos_newfull=np.cos(theta_newfull) #would be the best, but bisplev requires a linear'''
cos_newfull, phi_newfull = np.mgrid[-0.999:0.999:90j, -179.9:179.9:90j]
# cos_newfull, phi_newfull = np.mgrid[-0.9875:0.9875:90j, -177.25:177.25:90j]

# phicos_PHOTON_full = list(itertools.product(np.linspace(-177.25,177.25,nbin_density),np.linspace(-0.9875,0.9875,nbin_density)))
phicos_PHOTON_full = list(itertools.product(np.linspace(-179.9,179.9,nbin_density),np.cos(np.linspace(-np.pi-0.02,0.-0.02,nbin_density))))

cos_fullM=np.array([el[1] for el in phicos_PHOTON_full])
phi_fullM=np.array([el[0] for el in phicos_PHOTON_full])

theta_rad_full = np.arccos(cos_fullM)
phi_rad_full = phi_fullM/180.*np.pi

In [ ]:
plt.plot(theta_rad_full,".")

In [ ]:
plt.plot(cos_full,"-",c="r")
plt.plot(cos_newM,"-",c="k")
plt.plot(cos_fullM,"-",c="y")
plt.plot(cos_new[:,0],"-",c="b")

In [ ]:
plt.plot(phi_full,"-",c="r")
plt.plot(phi_newM,"-",c="k")
plt.plot(phi_fullM,"-",c="y")
plt.plot(phi_new[0,:],"-",c="b")

In [ ]:
#insert hier the corrispondent index number coming from the comparison with the b1 map values
#NOTE: change the output name of the lod for the mesh and the right s factor in bisplrep
checkthis=37
# checkthis=4
MFPAD_totordered_phi_norm[checkthis].max()

#look to the values in b1 map and found the MFPAD. The phi - cos(theta) grid of light direction id indipendendt from the enantiomer
#the number of the element depends on the sorting of the b1 map at which the maximum has been calculated. Usually is phi sorted.
'''' WE USE cosphi_adj_phi, but cosphi_photon could be used too.'''
dfind=pd.DataFrame(cosphi_adj_phi, columns=["ctheta","phi"])
dfind.loc[dfind["phi"] == 105]

In [ ]:
#fit the data with the bspline, produce a much denser cloud!
from scipy import interpolate
theta_hi=np.linspace(0.,np.pi,18*2); phi_hi=np.linspace(0.,2*np.pi,36*2);

#the following has to be linear in theta, but data have neem produced linear in cos 
# f0 = interpolate.SmoothSphereBivariateSpline(theta_rad, phi_rad+np.pi, MFPAD_tot_sum_norm.reshape(-1), s=6.9)
# f0 = interpolate.SmoothSphereBivariateSpline(theta_rad, phi_rad+np.pi, MFPAD_totordered_phi_norm[checkthis].reshape(-1), s=6.5)
# r0 = f0(theta_hi,phi_hi).T

'''rbf = interpolate.Rbf(cos_full, phi_full, MFPAD_totordered_phi_norm[checkthis].T,function='multiquadric',smooth=0.1)
r3 = rbf(cos_fullM, phi_fullM) #this would prbably be the best for scattered sample'''

ftot = interpolate.bisplrep(cos_full, phi_full, MFPAD_tot_sum_norm.reshape(-1), s=0.7) #MFPAD_tot
f37 = interpolate.bisplrep(cos_full, phi_full, MFPAD_totordered_phi_norm[checkthis].reshape(-1), s=8.5) #MFPAD37
f4 = interpolate.bisplrep(cos_full, phi_full, MFPAD_totordered_phi_norm[4].reshape(-1), s=8.5) #MFPAD4
f0 = interpolate.bisplrep(cos_full, phi_full, MFPAD_totordered_phi_norm[0].reshape(-1), s=4.8) #MFPAD0

rtot_full = interpolate.bisplev(cos_newfull[:,0], phi_newfull[0,:],ftot).T
r37_full = interpolate.bisplev(cos_newfull[:,0], phi_newfull[0,:],f37).T
r4_full = interpolate.bisplev(cos_newfull[:,0], phi_newfull[0,:],f4).T
r0_full = interpolate.bisplev(cos_newfull[:,0], phi_newfull[0,:],f0).T

rtot = interpolate.bisplev(cos_new[:,0], phi_new[0,:],ftot).T
r37 = interpolate.bisplev(cos_new[:,0], phi_new[0,:],f37).T
r4 = interpolate.bisplev(cos_new[:,0], phi_new[0,:],f4).T
r0 = interpolate.bisplev(cos_new[:,0], phi_new[0,:],f0).T

print(rtot.shape)
print(rtot_full.shape)
print(r37.shape)
print(r37_full.shape)

plt.imshow(r0.T, cmap='viridis')

In [ ]:
#tot MPFAD
fig,ax=plt.subplots()
titel_fig=enant+" MFPAD tot RAW"

#NOTE transposed with respect to imshow
# fig.pcolormesh(MFPAD_totordered_phi_norm[checkthis].T, cmap='viridis')
ax.pcolormesh(MFPAD_tot_sum.T, cmap='viridis')

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos(\u03D1) [adm]')
ax.set_title(titel_fig)

In [ ]:
#tot MPFAD
fig,ax=plt.subplots()
titel_fig=enant+" MFPAD tot smooth"

#NOTE transposed with respect to imshow
ax.imshow(rtot.reshape(nbin_density,nbin_density).T, cmap='viridis')

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos(\u03D1) [adm]')
ax.set_title(titel_fig)

In [ ]:
#tot MPFAD
fig,ax=plt.subplots()
titel_fig=enant+" MFPAD tot smooth full"

#NOTE transposed with respect to imshow
ax.imshow(rtot_full.reshape(nbin_density,nbin_density).T, cmap='viridis')

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos(\u03D1) [adm]')
ax.set_title(titel_fig)

In [ ]:
#single MPFAD
fig,ax=plt.subplots()
titel_fig=enant+" MFPAD el "+str(checkthis)+"_RAW"

#NOTE transposed with respect to imshow
ax.pcolormesh(MFPAD_totordered_phi_norm[checkthis].T, cmap='viridis')

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos(\u03D1) [adm]')
ax.set_title(titel_fig)

In [ ]:
#single MPFAD
fig,ax=plt.subplots()
titel_fig=enant+" MFPAD el smooth "+str(checkthis)

#NOTE transposed with respect to imshow
# ax.pcolormesh(r37.reshape(nbin_density,nbin_density).T, cmap='viridis', shading="flat")
ax.imshow(r37.reshape(nbin_density,nbin_density).T, cmap='viridis')

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos(\u03D1) [adm]')
ax.set_title(titel_fig)

In [ ]:
#single MPFAD
fig,ax=plt.subplots()
titel_fig=enant+" MFPAD el "+str(checkthis)+"_full"

#NOTE transposed with respect to imshow
# ax.pcolormesh(r37.reshape(nbin_density,nbin_density).T, cmap='viridis', shading="flat")
ax.imshow(r37_full.reshape(nbin_density,nbin_density).T, cmap='viridis')

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos(\u03D1) [adm]')
ax.set_title(titel_fig)

In [ ]:
#total MFPAD RAW
#from spherical to cartesian coordinates

p=len(phiM); c=len(cosM)

counts_original=MFPAD_tot_sum.reshape(-1)
xraw = counts_original * np.sin(theta_rad) * np.cos(phi_rad)
Xraw = xraw.reshape(p,c)
yraw = counts_original * np.sin(theta_rad) * np.sin(phi_rad)
Yraw = yraw.reshape(p,c)
zraw = counts_original * np.cos(theta_rad)
Zraw = zraw.reshape(p,c)

draw = np.sqrt(xraw**2+yraw**2+zraw**2)
draw_matrix = np.sqrt(Xraw**2+Yraw**2+Zraw**2)

In [ ]:
#tot MFPAD full
#from spherical to cartesian coordinates

counts=rtot_full.reshape(-1)
# counts=rtot.reshape(-1) #standars domain for cos and phi
x = counts * np.sin(theta_rad_new.ravel()) * np.cos(phi_rad_new.ravel())
X = x.reshape(nbin_density,nbin_density)
y = counts * np.sin(theta_rad_new.ravel()) * np.sin(phi_rad_new.ravel())
Y = y.reshape(nbin_density,nbin_density)
z = counts * np.cos(theta_rad_new.ravel())
Z = z.reshape(nbin_density,nbin_density)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
#single MFPAD to check
#from spherical to cartesian coordinates

countsf=r37.reshape(-1)
x37 = countsf * np.sin(theta_rad_full.ravel()) * np.cos(phi_rad_full.ravel())
X37 = x37.reshape(nbin_density,nbin_density)
y37 = countsf * np.sin(theta_rad_full.ravel()) * np.sin(phi_rad_full.ravel())
Y37 = y37.reshape(nbin_density,nbin_density)
z37 = countsf * np.cos(theta_rad_full.ravel())
Z37 = z37.reshape(nbin_density,nbin_density)

df = np.sqrt(x37**2+y37**2+z37**2)
df_matrix = np.sqrt(X37**2+Y37**2+Z37**2)

In [ ]:
fig1 = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                    marker=dict( size=5,color=d, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

# fig1.update_layout(title="TFMOx MFPAD el "+str(checkthis), width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig1.show()

fig2 = go.Figure(data=[go.Scatter3d(x=xf, y=yf, z=zf, mode='markers',
                    marker=dict( size=5,color=df, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

# fig2.update_layout(title="TFMOx MFPAD el "+str(checkthis)+" fixed", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig2.show()

fig3 = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs, mode='markers',
                    marker=dict( size=5,color=ds, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

fig3.update_layout(title="TFMOx MFPAD tot", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig3.show()

delaunay_tri, point_trace, my_mesh3d, delaun_tri3d = fugi.makeamesh(xs,ys,zs,ds)

fig2 = go.Figure(data=[delaunay_tri, point_trace])
fig2.update_layout(width=500, height=500);
#fig2.show()

my_layout = dict(width=800, height=600,
                 scene_camera_eye=dict(x=1.5, y=1.5, z=1), 
                 scene_aspectratio=dict(x=1.5, y=1.5, z=1))

fig2s = go.Figure(data=[my_mesh3d, delaun_tri3d])
#fig2s.show()

xn,yn,zn,In,Jn,Kn = fugi.getamesh(x,y,z,d)

fig3 = go.Figure(go.Mesh3d(x=x, y=y, z=z,
                           alphahull=5,  
                        #    i=In, j=Jn, k=Kn, 
                           intensity=d,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig3.update_layout(width=800, height=800)
# fig1.show()


fig4 = go.Figure(data=[go.Surface(z=Z, surfacecolor=d_matrix)])
fig4.update_layout(title='TFMOx MFPADs EXP surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig4.show()

fig4s = go.Figure(data=[go.Surface(z=Zs, surfacecolor=d_matrixs)])
fig4s.update_layout(title='TFMOx MFPADs EXP surf SMOOTHED', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig4s.show()


import open3d as o3d
point_cloud=np.array([xf,yf,zf]).T #Standard
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)

o3d.io.write_point_cloud("../PYTHON_graphs/OUTPUTS/sync.ply", pcd)

# Load saved point cloud and visualize it
pcd_load = o3d.io.read_point_cloud("../PYTHON_graphs/OUTPUTS/sync.ply")
# Convert Open3D.o3d.geometry.PointCloud to numpy array
xyz_load = np.asarray(pcd_load.points)
print('xyz_load')
print(xyz_load)
o3d.visualization.draw_geometries([pcd_load])

In [ ]:
if n2 == True:
    # import variables from 3D_MFPDAs_theory!
    %store -r phi_full_n2
    %store -r phi_mol2D
    %store -r ctheta_full_n2
    %store -r ctheta_mol
    %store -r MFPAD3D
    
    MFPAD3D_norm=fugi.normalise_with_err(MFPAD3D,normtype=3)
    
    phi_n2=np.linspace(-180,180,1728)
    ctheta_n2=np.linspace(-1,1,1728)
    
    cos_newfulln2, phi_newfulln2 = np.mgrid[-1:1:90j, -180:180:90j]
    
    phicos_n2 = list(itertools.product(np.linspace(-180,180,nbin_density),np.cos(np.linspace(np.arccos(-1),np.arccos(1),nbin_density))))
    
    cos_new_n2=np.array([el[1] for el in phicos_n2])
    phi_new_n2=np.array([el[0] for el in phicos_n2])
    
    theta_rad_n2 = np.arccos(cos_new_n2)
    phi_rad_n2 = phi_new_n2/180.*np.pi
    
    fn2 = interpolate.bisplrep(ctheta_full_n2, phi_full_n2, (MFPAD3D_norm.T).reshape(-1),s=0.5)
    r1n2 = interpolate.bisplev(cos_newfulln2[:,0], phi_newfulln2[0,:],fn2).T
    
    counts=r1n2.reshape(-1)
    x_n2 = counts * np.sin(theta_rad_n2.ravel()) * np.cos(phi_rad_n2.ravel())
    X_n2 = x_n2.reshape(nbin_density,nbin_density)
    y_n2 = counts * np.sin(theta_rad_n2.ravel()) * np.sin(phi_rad_n2.ravel())
    Y_n2 = y_n2.reshape(nbin_density,nbin_density)
    z_n2 = counts * np.cos(theta_rad_n2.ravel())
    Z_n2 = z_n2.reshape(nbin_density,nbin_density)
    
    d_n2 = np.sqrt(x_n2**2+y_n2**2+z_n2**2)
    d_matrix_n2 = np.sqrt(X_n2**2+Y_n2**2+Z_n2**2)

    plt.imshow(r1n2.T, cmap='viridis')

In [ ]:
def lod_mesh_export(mesh, lods, extension, path, enant):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        if enant == "R":
            o3d.io.write_triangle_mesh(path+"R_lod_"+str(i)+extension, mesh_lod)
        else:
            o3d.io.write_triangle_mesh(path+"S_lod_"+str(i)+extension, mesh_lod) #for 11.5 eV
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
import open3d as o3d
import trimesh

sys.path.append('..'); output_path=(r"../PYTHON_graphs/OUTPUTS/meshes/"+en+"/")
alpha_mesh=[]; poisson_mesh=[]; densities=[]

alpha_mesh = o3d.geometry.TriangleMesh()
pcd = o3d.geometry.PointCloud()
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals
camera_loc=np.array([0.,0.,-1.])

#load the point cloud: select the desired point cloud
# point_cloud=np.array([xf,yf,zf]).T #single MFPAD -> chanhe the number in the outputfile (e.g. 37 or 4)
# point_cloud=np.array([x,y,z]).T #single MFPAD -> chanhe the number in the outputfile (e.g. 37 or 4)
point_cloud=np.array([xs,ys,zs]).T #sum MFPAD

cloud = PyntCloud.from_instance("open3d", pcd)
pcd.points = o3d.utility.Vector3dVector(point_cloud)
#resise the scale of the sample
vox_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, 1.)

#KD neighbour
kdtree = cloud.add_structure("kdtree")
# dela3d = cloud.add_structure("delaunay3D")
testc = cloud.get_neighbors(k=10)
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)

#compute the normals
pcd.estimate_normals(); #mandatory
# mesh.compute_vertex_normals()
# pcd.normals = o3d.utility.Vector3dVector(point_cloud) #perfectly radial and long, independent by the tangent plane
# They seem good for open surfaces, but not for my case
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05,max_nn=20));
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50));

'''orient the normals'''
#Number of nearest neighbours: 5 is the minimum to have a closed surface with scale >= 2
pcd.orient_normals_consistent_tangent_plane(7) #higher #bins
# pcd.orient_normals_consistent_tangent_plane(5) #normal bins
# pcd.orient_normals_towards_camera_location(camera_loc) #normal bins
# pcd.orient_normals_to_align_with_direction([0.,1.,0.]) #normal bins

'''####################################################################
#########################alpha-shape algorithm#########################
####################################################################'''
# alpha=np.log10(0.5)
# alpha=0.1
# tetralpha_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)
# alpha_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha, tetralpha_mesh, pt_map) #it resturns zero for some reason

'''###############################################################
###################### Poisson algorithm #########################
###############################################################'''
#high bins
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, width=0, scale=2.5, linear_fit=False)
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=2., linear_fit=False)
#normal bins single MFAD
poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=1.8, linear_fit=False)
#normal bins tot MFPAD
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, width=0, scale=1.2, linear_fit=False)
#normal bins N2 MFPAD
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=7, width=0, scale=1.2, linear_fit=False)

bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox) #SOMETIMES cropping could damage the mesh
# p_mesh_crop.compute_triangle_normals() # usually computed before rendering (??)

'''################################################################
####################### Hull ball algoritm ########################
###############################################################'''
# radii = [0.08, 0.1, 0.2, 0.4] #multiple radii!
# bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector(radii))

#both for single and tot MFPAD
radius = 1.5 * avg_dist
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

if n2 == True:
    radius = 5.5 * avg_dist
    bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 4]))

'''
CLEANING THE TRIANGLES
arget_number_of_triangles (int) – The number of triangles that the simplified mesh should have. It is not guaranteed that this number will be reached.
maximum_error (float, optional, default=inf) – The maximum error where a vertex is allowed to be merged
boundary_weight (float, optional, default=1.0) – A weight applied to edge vertices used to preserve boundaries
'''
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

#none of the followings is working with poisson
# p_mesh_crop =poisson_mesh.simplify_quadric_decimation(6000)
# p_mesh_crop.remove_unreferenced_vertices
# p_mesh_crop.remove_degenerate_triangles()
# p_mesh_crop.remove_duplicated_triangles()
# p_mesh_crop.remove_duplicated_vertices()
# p_mesh_crop.remove_non_manifold_edges()

'''create the triangular mesh with the vertices and faces from open3d to be visualise with another software'''
tri_mesh = trimesh.Trimesh(np.asarray(dec_mesh.vertices), np.asarray(dec_mesh.triangles),vertex_normals=np.asarray(dec_mesh.vertex_normals))
tri_mesh_pois = trimesh.Trimesh(np.asarray(poisson_mesh.vertices), np.asarray(poisson_mesh.triangles),vertex_normals=np.asarray(poisson_mesh.vertex_normals))
# tri_mesh_pois_crop = trimesh.Trimesh(np.asarray(p_mesh_crop.vertices), np.asarray(p_mesh_crop.triangles),vertex_normals=np.asarray(p_mesh_crop.vertex_normals))
# trimesh.convex.is_convex(dec_mesh)

In [ ]:
# np.asarray(poisson_mesh.vertices).T
xgo, ygo, zgo =np.asarray(poisson_mesh.vertices).T
dgo = np.sqrt(xgo**2+ygo**2+zgo**2)

# np.asarray(poisson_mesh.vertex_normals)
# poisson_mesh.triangle_normals
tigo, tjgo, tkgo =np.asarray(poisson_mesh.triangles).T
ti, tj, tk = tri_mesh_pois.triangles.T

# asarray(p_mesh_crop.triangles)
# txgo, tygo, tzgo =np.asarray(p_mesh_crop.vertices).T[0]
# tdgo = np.sqrt(txgo**2+tygo**2+tzgo**2)

In [ ]:
#designing the surface colour; densities are the real density of features from o3d
#NOTE chose an output name between MFPADtot, MFPAD0, MFPAD4
densities = np.asarray(densities)
density_colors = plt.get_cmap('viridis')((dgo - dgo.min()) / (dgo.max() - dgo.min()))
density_colors = density_colors[:, :3]

#works for the plotting in o3d
poisson_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
bpa_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
tri_mesh_pois.vertex_colors = o3d.utility.Vector3dVector(density_colors)

if enant == "R":
    o3d.io.write_triangle_mesh(output_path+"R_MFPAD_tot_bpa_mesh.ply", dec_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_37_bpa_mesh.ply", dec_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_4_bpa_mesh.ply", dec_mesh);
    o3d.io.write_triangle_mesh(output_path+"R_MFPAD_tot_p_mesh.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_37_p_mesh.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_4_p_mesh.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_p_mesh_c.ply", p_mesh_crop);  #SOMETIMES cropping could damage the mesh
else:
    o3d.io.write_triangle_mesh(output_path+"S_MFPAD_tot_bpa_mesh.ply", dec_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_37_bpa_mesh.ply", dec_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_4_bpa_mesh.ply", dec_mesh);
    o3d.io.write_triangle_mesh(output_path+"S_MFPAD_tot_p_mesh.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_37_p_mesh.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_4_p_mesh.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_p_mesh_c.ply", p_mesh_crop);  #SOMETIMES cropping could damage the mesh

if n2 == True:
    my_lods = lod_mesh_export(p_mesh_crop, [100000,50000,10000,1000,100], "_N2.ply", output_path)

# my_lods = lod_mesh_export(p_mesh_crop, [100000,50000,10000,1000,100], ".ply", output_path, enant)
my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], "_MFPAD_tot.ply", output_path, enant)
# my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], "_MFPAD_37.ply", output_path, enant)
# my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], "_MFPAD_4.ply", output_path, enant)
# my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], "_MFPAD_0.ply", output_path, enant)


In [ ]:
# o3d.visualization.draw_geometries([pcd, p_mesh_crop], mesh_show_back_face=True)
# o3d.visualization.draw_geometries([pcd, bpa_mesh], mesh_show_back_face=True)
o3d.visualization.draw_geometries([pcd, poisson_mesh],mesh_show_back_face=True)
# o3d.visualization.draw_geometries([pcd, poisson_mesh[100000]],point_show_normal=True)

In [ ]:
# tri_mesh.show()
tri_mesh_pois.show()

In [ ]:
fig5 = go.Figure(go.Mesh3d(x=xgo, y=ygo, z=zgo, 
                           # alphahull=3.5, 
                           i=tigo, j=tjgo, k=tkgo, 
                           # i=ti, j=tj, k=tk, 
                           intensity=dgo,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig5.update_layout(width=800, height=800)
fig5.show()
# plotly.offline.plot(fig5, filename=r'../PYTHON_graphs/OUTPUTS/MFPAD4.html')


#Go scatters do not provide a legend
fig6 = go.Figure(data=[go.Scatter3d(x=xgo, y=ygo, z=zgo,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=dgo,            #set color to an array/list of desired values
                        colorscale='Viridis',   #choose a colorscale
                        opacity=1.,
                        showscale=True          #to show the legend according to the color
                   )
                )])

fig6.update_layout(title="TFMOx MFPAD poisson scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #margin=dict(l=0, r=0, b=0, t=0))
fig6.show()

#fig6 = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs,
fig6s = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        #color=ds,
                        color=d,
                        colorscale='Viridis',   #choose a colorscale
                        opacity=1.,
                        showscale=True          #to show the legend according to the color
                   )
                )])

fig6s.update_layout(title="TFMOx MFPAD smoothed scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #margin=dict(l=0, r=0, b=0, t=0))

fig6s.show()

In [ ]:
#SHORTCUTS from keyboard: n = show normals, q = quit, w = mesh
# o3d.visualization.draw_geometries([pcd, poisson_mesh],mesh_show_back_face=True)

In [ ]:
def rot3d(alpha,beta,gamma):  #planar rotation of alpha radians
    return np.array([[np.cos(alpha)*np.cos(beta), np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)], 
                      [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
                      [-np.sin(beta),              np.cos(beta)*np.sin(gamma),                                           np.cos(beta)*np.cos(gamma)]])

frames=[]

fig8 = go.Figure(go.Mesh3d(x=xgo, y=ygo, z=zgo,i=tigo, j=tjgo, k=tkgo, intensity=dgo,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05,facenormalsepsilon=0,vertexnormalsepsilon=0),lightposition=dict(x=100,y=100,z=1000)))

T = np.arange(0,  2*np.pi, 0.125)
xyz = np.stack((xgo, ygo, zgo))
for alpha in T:
    xr, yr, zr  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyz)#  batch rotation of all points (x,y,z) on the Mesh
    frames.append(go.Frame(data=[go.Mesh3d(x=xr, y=yr, z=zr)])) #z is the same in a 3d rotation about zaxis
fig8.update(frames=frames);    


fig8.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

fig8.update_layout(updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=0.7,
                  x=-0.15,
                  xanchor='left',
                  yanchor='bottom',
                  buttons=[dict(label='Play',
                                 method='animate',
                                 args=[None, dict(frame=dict(duration=5, redraw=True), 
                                                             transition=dict(duration=4),
                                                             fromcurrent=True,
                                                            #  mode='immediate'
                                                            )]
                                            )
                                      ]
                              )
                        ])

# fig8.show()
# plotly.offline.plot(fig8, filename=r'../PYTHON_graphs/OUTPUTS/3D_exp_th_frames_n_highbins2.html')

# import variables from 3D_MFPDAs_theory!
%store -r X
%store -r Y
%store -r Z
xth=X.reshape(-1)
yth=Y.reshape(-1)
zth=Z.reshape(-1)
d_matrix = np.sqrt(X**2+Y**2+Z**2)
dth=d_matrix.reshape(-1)

frames=[]

camera = dict(
    up=dict(x=0, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=0)
)


fig9 = go.Figure()

# fig9.update_scenes(camera=camera)


fig9 = make_subplots(rows=1, cols=2,
                     subplot_titles=('Experiment', 'Theory'),
                     specs=[[{"type": "mesh3d"}, {"type": "surface"}]],)


fig9.add_trace(go.Mesh3d(x=xgo, y=ygo, z=zgo,i=tigo, j=tjgo, k=tkgo, intensity=dgo,
                        coloraxis="coloraxis1",
                        #    colorscale="Viridis",
                     #       colorbar_len=0.75,
                     #       flatshading=True,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05,facenormalsepsilon=0,vertexnormalsepsilon=0),lightposition=dict(x=100,y=100,z=1000)),1,1)
        
fig9.add_trace(go.Surface(x=X, y=Y, z=Z,
                          surfacecolor=d_matrix,
                          connectgaps=True,
                        coloraxis="coloraxis2",

                        #   colorscale="Viridis",
                     #       colorbar_len=0.75,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05),lightposition=dict(x=100,y=100,z=1000)),1,2)



T = np.arange(0,  2*np.pi, 0.125)
# T = np.arange(0,  2*np.pi, 0.0625)
xyzm = np.stack((xgo, ygo, zgo))
xyzs = np.stack((X.reshape(-1), Y.reshape(-1), Z.reshape(-1)))
for alpha in T:
    xrm, yrm, zrm  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyzm)#  batch rotation of all points (x,y,z) on the Mesh
    xrs, yrs, zrs  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyzs)#  batch rotation of all points (x,y,z) on the Mesh
    dths= np.sqrt(xrs**2+xrs**2+zrs**2)
    frames.append(go.Frame(data=[go.Mesh3d(x=xrm, y=yrm, z=zrm),
                                 go.Surface(x=xrs.reshape(200,100), 
                                            y=yrs.reshape(200,100),
                                            z=zrs.reshape(200,100),
                                            surfacecolor=d_matrix)],
                           traces=[0,1])) 
fig9.update(frames=frames)

fig9.update_layout(scene=dict(xaxis_range=[-46,46], xaxis_visible=False, xaxis_autorange=False,
                              yaxis_range=[-46,46], yaxis_visible=False, yaxis_autorange=False,
                              zaxis_range=[-46,46], zaxis_visible=False, zaxis_autorange=False),
                  scene2=dict(xaxis_range=[-0.007, 0.007], xaxis_visible=False, xaxis_autorange=False,
                              yaxis_range=[-0.007, 0.007], yaxis_visible=False, yaxis_autorange=False,
                              zaxis_range=[-0.007, 0.007], zaxis_visible=False, zaxis_autorange=False),
                  coloraxis1=dict(colorscale='Viridis', colorbar=dict(x=-.1, xanchor='left', thickness=20)),
                  coloraxis2=dict(colorscale='Viridis', colorbar=dict(x=1.05, xanchor='right', thickness=20)),

                  margin=dict(l=0, r=0, t=0, b=0),

                  # scene2=dict(xaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False),
                  #             yaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False),
                  #             zaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False)),
                                             
                  updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=0.7,
                  x=-0.15,
                  xanchor='left',
                  yanchor='bottom',
                  buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=0, redraw=True), 
                                                                     transition=dict(duration=0,easing="quadratic-in-out"),
                                                                     fromcurrent=True,
                                                                  #    mode='immediate'
                                                                  )]),
                           dict(label='Pause', method='animate', args=[None, dict(frame=dict(duration=0, redraw=False), 
                                                                      mode='immediate')])


                          ]
                      )]
                  )

# fig9.show()
plotly.offline.plot(fig9, filename=r"../PYTHON_graphs/OUTPUTS/3D_"+CH+"_"+en+"_exp.html")

print(fig9.layout)